# Extração da Drogaria Minas Mais

In [21]:
import requests
from bs4 import BeautifulSoup
import os
import time
import json
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import regex as re

In [19]:
# Capturando todas as URLs

# Função de acessar site
all_workers = os.cpu_count()

def access_site(url, max_attempts=all_workers):
    headers = {'Accept': '*/*', 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    for retry in range(max_attempts):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
        except Exception as e:
                print(f"An error occurred while accessing URL: {url}: {e}. Retrying... (Attempt {retry + 1}/{max_attempts})")
                time.sleep(1)

# Função captura de todos os nomes de marcas e criação de URLs
def getNomeMarcas():
    response = access_site("https://www.drogariasminasmais.com.br/medicamentos")
    soup = BeautifulSoup(response.text, 'html.parser')
    sessaoMarcasGrandes = soup.find('div', class_='vtex-search-result-3-x-filter__container vtex-search-result-3-x-filter__container--responsive-result-content-mz0001 bb b--muted-4 vtex-search-result-3-x-filter__container--brand')
    sessaoMarcas = sessaoMarcasGrandes.find('div', class_='vtex-search-result-3-x-filterTemplateOverflow vtex-search-result-3-x-filterTemplateOverflow--responsive-result-content-mz0001 pb5 overflow-y-auto')
    marcas = sessaoMarcas.find_all('label', class_='vtex-checkbox__label w-100 c-on-base pointer')
    return marcas

# Criando uma lista de URLs ainda não bem estruturadas
urlSuja = getNomeMarcas()
urlLimpa = []

# Limpando lista das URLs
for i in urlSuja:
    # Criando texto separado e minusculo com "-"
    texto = i.text.lower()
    textoNovo = texto.replace(" ", "-")

    # Salvando tudo em uma lista
    urlLimpa.append(textoNovo)

listaUrls = []

# Criando todas as URLs
for j in urlLimpa:
    URL = "https://www.drogariasminasmais.com.br/medicamentos/" + j + "?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page="
    listaUrls.append(URL)


In [20]:
# Mudando o número das páginas

# Função para encontrar um arquivo JSON secreto dependendo da página e da marca
def getJSON(urlInicial, page_number):
    url = str(urlInicial) + str(page_number)
    requisicao = requests.get(url)
    soup = BeautifulSoup(requisicao.text, 'html.parser')
    jsonSujo = soup.find_all('script')[10]
    print(jsonSujo)
    return jsonSujo

total_pages = 25
listaJSON = []

# Iteração por páginas de usuários
for i in listaUrls: 
    for page_number in range(1, total_pages + 1):
        listaJSON.append(getJSON(i, page_number))
        print("---------------------------------------------------------------")

listaJSON

<script crossorigin="" src="https://io.vtex.com.br/v3/polyfill.min.js?version=3.89.4&amp;features=Object.values,Object.entries,Object.assign,Array.from,Array.prototype.findIndex,Array.prototype.fill,Array.prototype.find,Array.prototype.flatMap,Array.prototype.includes,String.prototype.startsWith,String.prototype.endsWith,String.prototype.includes,String.prototype.repeat,Symbol,fetch,Promise,Map,WeakMap,Number.isInteger,Number.isNaN,Number.EPSILON,Set,Function.name,URL,IntersectionObserver,Intl.~locale.pt-BR&amp;flags=gated"></script>
---------------------------------------------------------------
<script crossorigin="" src="https://io.vtex.com.br/v3/polyfill.min.js?version=3.89.4&amp;features=Object.values,Object.entries,Object.assign,Array.from,Array.prototype.findIndex,Array.prototype.fill,Array.prototype.find,Array.prototype.flatMap,Array.prototype.includes,String.prototype.startsWith,String.prototype.endsWith,String.prototype.includes,String.prototype.repeat,Symbol,fetch,Promise,Ma

KeyboardInterrupt: 

In [26]:
# Definindo váriaveis necessarias para o selenium
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
options = webdriver.ChromeOptions()

options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless=new')

navegador = webdriver.Chrome(options=options)
wait = WebDriverWait(navegador, 3)

/bin/sh: 1: google-chrome: not found
/bin/sh: 1: google-chrome-stable: not found
/bin/sh: 1: google-chrome-beta: not found
/bin/sh: 1: google-chrome-dev: not found
/bin/sh: 1: google-chrome: not found
/bin/sh: 1: google-chrome-stable: not found
/bin/sh: 1: google-chrome-beta: not found
/bin/sh: 1: google-chrome-dev: not found


AttributeError: 'NoneType' object has no attribute 'split'

In [25]:
# Mudando o número das páginas

# Função para encontrar um arquivo JSON secreto dependendo da página e da marca
def getJSON(urlInicial, page_number):
    url = str(urlInicial) + str(page_number)
    driver.get(url)
    jsonSujo = wait.until(EC.visibility_of_element_located((By.XPATH,'/html/body/div[2]/div/div[1]/div/div[2]/div/div/section/div[2]/div/div[3]/div/div[2]/div/div[2]/div/div/div/div/div[1]/script/text()')))
    return jsonSujo

total_pages = 25
listaJSON = []

# Iteração por páginas de usuários
for i in listaUrls: 
    for page_number in range(1, total_pages + 1):
        listaJSON.append(getJSON(i, page_number))

driver.quit()
listaJSON

NameError: name 'driver' is not defined

In [16]:
from selenium import webdriver

driver = webdriver.Chrome(service=r"./chomedriver.exe")

url = "https://www.google.com/"
driver.get(url)

driver.quit()

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [ ]:
# Capturando todos os dados dentro do JSON

In [ ]:
# Enviando tudo para um CSV
